In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import numpy as np
from torch.autograd import Variable
#from utils import plot_images
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:

# Hyper Parameters 
input_size = 784
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [3]:
def get_train_valid_loader(data_dir,
                           batch_size,
                           augment,
                           random_seed,
                           valid_size=0.1,
                           shuffle=True,
                           show_sample=False,
                           num_workers=4,
                           pin_memory=False):
    """
    Utility function for loading and returning train and valid
    multi-process iterators over the CIFAR-10 dataset. A sample
    9x9 grid of the images can be optionally displayed.
    If using CUDA, num_workers should be set to 1 and pin_memory to True.
    Params
    ------
    - data_dir: path directory to the dataset.
    - batch_size: how many samples per batch to load.
    - augment: whether to apply the data augmentation scheme
      mentioned in the paper. Only applied on the train split.
    - random_seed: fix seed for reproducibility.
    - valid_size: percentage split of the training set used for
      the validation set. Should be a float in the range [0, 1].
    - shuffle: whether to shuffle the train/validation indices.
    - show_sample: plot 9x9 sample grid of the dataset.
    - num_workers: number of subprocesses to use when loading the dataset.
    - pin_memory: whether to copy tensors into CUDA pinned memory. Set it to
      True if using GPU.
    Returns
    -------
    - train_loader: training set iterator.
    - valid_loader: validation set iterator.
    """
    error_msg = "[!] valid_size should be in the range [0, 1]."
    assert ((valid_size >= 0) and (valid_size <= 1)), error_msg

    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )

    # define transforms
    valid_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
    ])
    if augment:
        train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])
    else:
        train_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])

    # load the dataset
    train_dataset = dsets.MNIST(
        root=data_dir, train=True,
        download=True, transform=train_transform,
    )

    print len(train_dataset)
    valid_dataset = dsets.MNIST(
        root=data_dir, train=True,
        download=True, transform=valid_transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )

    # visualize some images
    if show_sample:
        sample_loader = torch.utils.data.DataLoader(
            train_dataset, batch_size=9, shuffle=shuffle,
            num_workers=num_workers, pin_memory=pin_memory,
        )
        data_iter = iter(sample_loader)
        images, labels = data_iter.next()
        X = images.numpy().transpose([0, 2, 3, 1])
        plot_images(X, labels)

    return (train_loader, valid_loader)


In [4]:
# MNIST Dataset (Images and Labels)

train_loader,test_loader = get_train_valid_loader('./data',
                           1,
                           False,
                           1,
                           valid_size=0.1,
                           shuffle=True,
                           show_sample=False,
                           num_workers=1,
                           pin_memory=True)
#train_dataset = dsets.MNIST(root='./data',train=True, transform=transforms.ToTensor(),download=True)
print len(test_loader)
print len(train_loader)

test_final_dataset = dsets.MNIST(root='./data',train=False, transform=transforms.ToTensor())

60000
6000
54000


In [5]:
print (train_loader.size())

AttributeError: 'DataLoader' object has no attribute 'size'

In [6]:
# Dataset Loader (Input Pipline)
#train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
#                                           batch_size=batch_size, shuffle=True)

test_final_loader = torch.utils.data.DataLoader(dataset=test_final_dataset, 
                                          batch_size=batch_size, shuffle=False)

In [7]:
# Model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        out = self.linear(x)
        return out

model = LogisticRegression(input_size, num_classes)

In [8]:
# Loss and Optimizer
# Softmax is internally computed.
# Set parameters to be updated.
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 

In [9]:
# Training the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' 
% (epoch+1, num_epochs, i+1, len(train_loader)//batch_size, loss.data[0]))

/usr/local/lib/python2.7/dist-packages/torch/cuda/__init__.py:97: UserWarning: 
    Found GPU0 GeForce 940MX which is of cuda capability 5.0.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


Epoch: [1/2], Step: [100/540], Loss: 4.0922
Epoch: [1/2], Step: [200/540], Loss: 0.1564
Epoch: [1/2], Step: [300/540], Loss: 2.3126
Epoch: [1/2], Step: [400/540], Loss: 0.0387
Epoch: [1/2], Step: [500/540], Loss: 0.9841
Epoch: [1/2], Step: [600/540], Loss: 3.5838
Epoch: [1/2], Step: [700/540], Loss: 0.1118
Epoch: [1/2], Step: [800/540], Loss: 0.6058
Epoch: [1/2], Step: [900/540], Loss: 0.4226
Epoch: [1/2], Step: [1000/540], Loss: 0.0811
Epoch: [1/2], Step: [1100/540], Loss: 0.2246
Epoch: [1/2], Step: [1200/540], Loss: 0.6033
Epoch: [1/2], Step: [1300/540], Loss: 0.2885
Epoch: [1/2], Step: [1400/540], Loss: 0.1419
Epoch: [1/2], Step: [1500/540], Loss: 0.7785
Epoch: [1/2], Step: [1600/540], Loss: 0.2981
Epoch: [1/2], Step: [1700/540], Loss: 0.0120
Epoch: [1/2], Step: [1800/540], Loss: 0.0007
Epoch: [1/2], Step: [1900/540], Loss: 0.2216
Epoch: [1/2], Step: [2000/540], Loss: 5.0303
Epoch: [1/2], Step: [2100/540], Loss: 0.0842
Epoch: [1/2], Step: [2200/540], Loss: 0.0435
Epoch: [1/2], Step:

Epoch: [1/2], Step: [18300/540], Loss: 0.0049
Epoch: [1/2], Step: [18400/540], Loss: 0.1467
Epoch: [1/2], Step: [18500/540], Loss: 0.0112
Epoch: [1/2], Step: [18600/540], Loss: 0.1055
Epoch: [1/2], Step: [18700/540], Loss: 0.0021
Epoch: [1/2], Step: [18800/540], Loss: 0.0004
Epoch: [1/2], Step: [18900/540], Loss: 0.1106
Epoch: [1/2], Step: [19000/540], Loss: 0.5662
Epoch: [1/2], Step: [19100/540], Loss: 0.0028
Epoch: [1/2], Step: [19200/540], Loss: 0.6719
Epoch: [1/2], Step: [19300/540], Loss: 0.0044
Epoch: [1/2], Step: [19400/540], Loss: 0.0231
Epoch: [1/2], Step: [19500/540], Loss: 0.1331
Epoch: [1/2], Step: [19600/540], Loss: 0.0183
Epoch: [1/2], Step: [19700/540], Loss: 0.0017
Epoch: [1/2], Step: [19800/540], Loss: 0.0029
Epoch: [1/2], Step: [19900/540], Loss: 0.0001
Epoch: [1/2], Step: [20000/540], Loss: 0.0295
Epoch: [1/2], Step: [20100/540], Loss: 0.0075
Epoch: [1/2], Step: [20200/540], Loss: 1.2554
Epoch: [1/2], Step: [20300/540], Loss: 0.0338
Epoch: [1/2], Step: [20400/540], L

Epoch: [1/2], Step: [36300/540], Loss: 0.0046
Epoch: [1/2], Step: [36400/540], Loss: 0.0044
Epoch: [1/2], Step: [36500/540], Loss: 0.0000
Epoch: [1/2], Step: [36600/540], Loss: 0.0085
Epoch: [1/2], Step: [36700/540], Loss: 0.0023
Epoch: [1/2], Step: [36800/540], Loss: 0.0006
Epoch: [1/2], Step: [36900/540], Loss: 0.0025
Epoch: [1/2], Step: [37000/540], Loss: 0.0286
Epoch: [1/2], Step: [37100/540], Loss: 0.0245
Epoch: [1/2], Step: [37200/540], Loss: 0.5761
Epoch: [1/2], Step: [37300/540], Loss: 0.0111
Epoch: [1/2], Step: [37400/540], Loss: 0.0190
Epoch: [1/2], Step: [37500/540], Loss: 0.1802
Epoch: [1/2], Step: [37600/540], Loss: 0.0010
Epoch: [1/2], Step: [37700/540], Loss: 0.0000
Epoch: [1/2], Step: [37800/540], Loss: 0.0027
Epoch: [1/2], Step: [37900/540], Loss: 0.3575
Epoch: [1/2], Step: [38000/540], Loss: 0.0646
Epoch: [1/2], Step: [38100/540], Loss: 2.0010
Epoch: [1/2], Step: [38200/540], Loss: 0.0672
Epoch: [1/2], Step: [38300/540], Loss: 0.0136
Epoch: [1/2], Step: [38400/540], L

Epoch: [2/2], Step: [300/540], Loss: 8.5069
Epoch: [2/2], Step: [400/540], Loss: 0.0176
Epoch: [2/2], Step: [500/540], Loss: 0.0000
Epoch: [2/2], Step: [600/540], Loss: 5.3242
Epoch: [2/2], Step: [700/540], Loss: 0.0030
Epoch: [2/2], Step: [800/540], Loss: 0.0000
Epoch: [2/2], Step: [900/540], Loss: 0.0025
Epoch: [2/2], Step: [1000/540], Loss: 0.0007
Epoch: [2/2], Step: [1100/540], Loss: 0.0000
Epoch: [2/2], Step: [1200/540], Loss: 0.4771
Epoch: [2/2], Step: [1300/540], Loss: 0.0000
Epoch: [2/2], Step: [1400/540], Loss: 1.7572
Epoch: [2/2], Step: [1500/540], Loss: 0.0002
Epoch: [2/2], Step: [1600/540], Loss: 0.0006
Epoch: [2/2], Step: [1700/540], Loss: 0.4065
Epoch: [2/2], Step: [1800/540], Loss: 0.0001
Epoch: [2/2], Step: [1900/540], Loss: 0.0129
Epoch: [2/2], Step: [2000/540], Loss: 0.2538
Epoch: [2/2], Step: [2100/540], Loss: 0.0000
Epoch: [2/2], Step: [2200/540], Loss: 0.0001
Epoch: [2/2], Step: [2300/540], Loss: 7.6448
Epoch: [2/2], Step: [2400/540], Loss: 0.0000
Epoch: [2/2], Ste

Epoch: [2/2], Step: [18400/540], Loss: 0.0000
Epoch: [2/2], Step: [18500/540], Loss: 0.0028
Epoch: [2/2], Step: [18600/540], Loss: 0.0085
Epoch: [2/2], Step: [18700/540], Loss: 0.7566
Epoch: [2/2], Step: [18800/540], Loss: 0.0434
Epoch: [2/2], Step: [18900/540], Loss: 0.0006
Epoch: [2/2], Step: [19000/540], Loss: 0.0140
Epoch: [2/2], Step: [19100/540], Loss: 0.0466
Epoch: [2/2], Step: [19200/540], Loss: 0.0003
Epoch: [2/2], Step: [19300/540], Loss: 0.0114
Epoch: [2/2], Step: [19400/540], Loss: 0.0000
Epoch: [2/2], Step: [19500/540], Loss: 0.0008
Epoch: [2/2], Step: [19600/540], Loss: 0.0019
Epoch: [2/2], Step: [19700/540], Loss: 7.1031
Epoch: [2/2], Step: [19800/540], Loss: 0.0000
Epoch: [2/2], Step: [19900/540], Loss: 0.0000
Epoch: [2/2], Step: [20000/540], Loss: 0.0000
Epoch: [2/2], Step: [20100/540], Loss: 2.9024
Epoch: [2/2], Step: [20200/540], Loss: 1.0420
Epoch: [2/2], Step: [20300/540], Loss: 0.0000
Epoch: [2/2], Step: [20400/540], Loss: 0.1720
Epoch: [2/2], Step: [20500/540], L

Epoch: [2/2], Step: [36400/540], Loss: 0.0000
Epoch: [2/2], Step: [36500/540], Loss: 1.0180
Epoch: [2/2], Step: [36600/540], Loss: 0.0000
Epoch: [2/2], Step: [36700/540], Loss: 0.0475
Epoch: [2/2], Step: [36800/540], Loss: 3.5548
Epoch: [2/2], Step: [36900/540], Loss: 0.0001
Epoch: [2/2], Step: [37000/540], Loss: 0.3113
Epoch: [2/2], Step: [37100/540], Loss: 0.0006
Epoch: [2/2], Step: [37200/540], Loss: 0.0444
Epoch: [2/2], Step: [37300/540], Loss: 0.0012
Epoch: [2/2], Step: [37400/540], Loss: 0.0159
Epoch: [2/2], Step: [37500/540], Loss: 0.0001
Epoch: [2/2], Step: [37600/540], Loss: 0.0073
Epoch: [2/2], Step: [37700/540], Loss: 0.0608
Epoch: [2/2], Step: [37800/540], Loss: 0.0142
Epoch: [2/2], Step: [37900/540], Loss: 1.1387
Epoch: [2/2], Step: [38000/540], Loss: 0.0001
Epoch: [2/2], Step: [38100/540], Loss: 0.0255
Epoch: [2/2], Step: [38200/540], Loss: 0.3631
Epoch: [2/2], Step: [38300/540], Loss: 0.0000
Epoch: [2/2], Step: [38400/540], Loss: 1.0565
Epoch: [2/2], Step: [38500/540], L

In [10]:
# Test the Model
correct = 0
total = 0
for images, labels in test_final_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))


Accuracy of the model on the 10000 test images: 86 %


In [ ]:
# Save the Model
torch.save(model.state_dict(), 'model.pkl')